# Test influence of mesh refinement on conditioning
This test focuses on the influence of various grid resolutions. The test will keep `mesh_args` constant (except from standard length scaling) so that relative sizes of meshes across scaling coefficients is the same. In that regard, this test is separate from an experimental setup where different mesh parameters are to be tested.

## Code

In [2]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.extend([
    '/home/haakon/mastersproject', 
    '/home/haakon/mastersproject/src/mastersproject'
])

import porepy as pp
import numpy as np
import pandas as pd

import GTS as gts

# Logging:
import logging
logging.basicConfig(level='INFO')

# General setup
For all test cases, we will use the following common parameters:
* `sz=20`
    * This sets `mesh_size_frac`, `mesh_size_min`, `mesh_size_bound` to `sz`\
    * The "hope" is that uniform mesh size introduce the least amount of bias between tests of varying number of fractures.
* scaling coefficients to be set according to the optimal results from `fracture complexity.ipynb`. That is, for a given number of fractures, we choose the according optimal coefficients, then find the conditioning for successive refinements of the mesh.

In [3]:
from porepy.utils.grid_refinement import refine_mesh_by_splitting
from GTS.test.test_scaling.test_scaling_util import (
    make_params_for_scaling, 
    report_condition_number,
)

In [14]:
def condition_refined_meshes(**kwargs):
    """ Compute condition number over successively refined meshes
    
    """
    # 1. Step: Generate the first setup. Extract network for refinement.
    params = make_params_for_scaling(**kwargs)
    setup = gts.ContactMechanicsBiotISC(params)
    setup.create_grid()
    
    # 1. Step: Generate a number of meshes
    #     Refer to setup.create_grid()
    network = setup._network
    gmsh_in_path = f"{setup.viz_folder_name}/gmsh_frac_file.geo"
    gmsh_out_path = f"{setup.viz_folder_name}/gmsh_frac_file"
    
    # Create the generator
    gb_generator = refine_mesh_by_splitting(
        in_file=gmsh_in_path,
        out_file=gmsh_out_path,
        dim=3,
        network=network,
    )
    
    nref = kwargs.get('nref', 3)
    gb_list = [next(gb_generator) for _ in range(nref)]
    
    # We won't do this test for >10k cells
    n_cells = [gb.num_cells() for gb in gb_list]
    if max(n_cells) > 20000:
        print(f"max cells: {max(n_cells)}")
        breakpoint()
#         raise ValueError
    
    # Run test for each grid
    results = pd.DataFrame()
    for gb in gb_list:
        setup = gts.ContactMechanicsBiotISC(params)
        setup.set_grid(gb)
        setup.prepare_simulation()
        max_elem, max_A_sum, min_A_sum = report_condition_number(setup)
        v = {
            "num_cells": gb.num_cells(),
            "max_elem": max_elem,
            "max_A_sum": max_A_sum,
            "min_A_sum": min_A_sum
        }
        results = results.append(v, ignore_index=True)
        
    results['ratio'] = results['max_A_sum'] / results['min_A_sum']
    
    return results
    

In [16]:
params = {
    "sz": 20,
    "ls": 0.005,
    "ss": 1e9,
    "shearzone_names": ["S1_1", "S1_2", "S1_3", "S3_1"],
    'nref': 2,
}

data = condition_refined_meshes(**params)

INFO:GTS.isc_modelling.setup:Preparing setup for simulation on 2020-04-15T14:15:03+02:00
INFO:GTS.isc_modelling.setup:Simulation parameters:
 {'_gravity_bc': False,
 '_gravity_bc_p': False,
 '_gravity_src': False,
 'bounding_box': {'xmax': 80,
                  'xmin': -20,
                  'ymax': 150,
                  'ymin': 50,
                  'zmax': 75,
                  'zmin': -25},
 'folder_name': PosixPath('/home/haakon/mastersproject/src/mastersproject/GTS/isc_modelling/results/test_fracture_complexity/test_1'),
 'length_scale': 0.005,
 'mesh_args': {'mesh_size_bound': 20,
               'mesh_size_frac': 20,
               'mesh_size_min': 20},
 'path_head': 'test_fracture_complexity/test_1',
 'scalar_scale': 1000000000.0,
 'shearzone_names': ['S1_1', 'S1_2', 'S1_3', 'S3_1'],
 'solver': 'direct',
 'source_scalar_borehole_shearzone': {'borehole': 'INJ1', 'shearzone': 'S1_2'},
 'stress': array([[-11393046.22034133,    597621.26708787,   1842573.57669909],
       [   59762

INFO:GTS.isc_modelling.contact_mechanics_biot:Closest cell found has distance: 407.357691
INFO:porepy.models.contact_mechanics_biot_model:Discretize
INFO:porepy.numerics.fv.mpsa:Split MPSA discretization into 1 parts
INFO:porepy.numerics.fv.biot:Done with subproblem 0. Elapsed time 29.46365523338318
INFO:porepy.models.contact_mechanics_biot_model:Done. Elapsed time 39.17884182929993
INFO:porepy.viz.exporter:Construct 3d grid information using numba
/home/haakon/porepy/src/porepy/viz/exporter.py:932: NumbaPerformanceWarning: np.dot() is faster on contiguous arrays, called on (array(float64, 1d, A), array(float64, 1d, C))
  angle = np.arccos(np.dot(normals[:, fi], reference))
INFO:GTS.isc_modelling.contact_mechanics_biot:Finished 'prepare_simulation' in 42.5933 secs
INFO:GTS.test.test_scaling.test_scaling_util:Max element in A 7.28e+08
INFO:GTS.test.test_scaling.test_scaling_util:Max 7.36e+08 and min 2.02e+00 A sum.


In [17]:
# new new data
data

,max_A_sum,max_elem,min_A_sum,num_cells,ratio
0,8.834593e+08,7.279481e+08,2.028076,1229.0,4.356146e+08
1,7.357677e+08,7.277643e+08,2.024506,9448.0,3.634307e+08
